# REINFORCE -> PPO

---

In this notebook, we will train PPO with Farama Foundation's Inverted Pendulum environment.

### 1. Import the Necessary Packages

In [1]:
import gymnasium as gym
# gym.logger.set_level(40) # suppress warnings (please remove if gives error)
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

### 2. Define the Architecture of the Policy

In [4]:
env = gym.make("InvertedPendulum-v4", render_mode="human")
# env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

action_space_dims = env.action_space.shape[0]
print('action_space_dims:', action_space_dims)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):
    # def __init__(self, s_size=4, h_size=16, a_size=2):
    def __init__(self, s_size=4, h_size=16, a_size=action_space_dims):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()

        return action.item(), m.log_prob(action)

observation space: Box(-inf, inf, (4,), float64)
action space: Box(-3.0, 3.0, (1,), float32)
action_space_dims: 1


### 3. Train the Agent with REINFORCE

In [7]:
policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def reinforce(env, n_episodes=2500, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []

    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state, info = env.reset(seed=42)

        for t in range(max_t):
            action, log_prob = policy.act(state)
            print('action: ', action)
            print('action.shape: ', action.shape)
            print('log_prob: ', log_prob)
            saved_log_probs.append(log_prob)
            state, reward, terminated, truncated, info = env.step(action)
            rewards.append(reward)

            if terminated or truncated:
                break

        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        discounts = [gamma**i for i in range(len(rewards)+1)]
        R = sum([a*b for a,b in zip(discounts, rewards)])
        
        policy_loss = []

        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)

        policy_loss = torch.cat(policy_loss).sum()

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

        if np.mean(scores_deque)>=500.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores

env = gym.make("InvertedPendulum-v4")
scores = reinforce(env)
env.close()

action:  0


AttributeError: 'int' object has no attribute 'shape'

### 4. Plot the Scores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Watch a Smart Agent!

In [ ]:
env = gym.make("InvertedPendulum-v4", render_mode="human")

state, info = env.reset(seed=42)

for t in range(1000):
    action, _ = policy.act(state)
    env.render()

    state, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break 

env.close()